In [ ]:
import os
import numpy as np
import pandas as pd
import mne
import scipy.io
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
from mne.decoding import CSP
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from mne.decoding import CSP
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
brainAdress = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5',
'FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3','CP1','P1','P3',
'P5','P7','P9','PO7','PO3','O1','Iz','Oz','POz','Pz','CPz','Fpz','Fp2',
'AF8','AF4','AFz','Fz','F2','F4','F6','F8','FT8','FC6','FC4','FC2',
'FCz','Cz','C2','C4','C6','T8','TP8','CP6','CP4','CP2','P2','P4',
'P6','P8','P10','PO8','PO4','O2']

numpy dataの読み込み

In [ ]:
day1_data = np.load('numpy_data/subject1_data1.npy')
day1_label = np.load('numpy_data/subject1_label1.npy')
day2_data = np.load('numpy_data/subject1_data2.npy')
day2_label = np.load('numpy_data/subject1_label2.npy')

データの結合

In [ ]:
brain_data = np.vstack([day1_data,day2_data])
label_data = np.hstack([day1_label,day2_label])
# brain_data = np.load('numpy_data/subject1_image_data1.npy')
# label_data = np.load('numpy_data/subject1_image_label1.npy')

In [ ]:
motor_brainAdress = [9,10,11,12,13,17,18,31,44,45,46,47,48,49,50,54,55]
data_numpy_task  = brain_data[:,motor_brainAdress,100*6:100*10]

In [ ]:
info_motorbrain = ['FC3','FC1','C1','C3','C5','CP3','CP1','CPz','FC4','FC2',
'FCz','Cz','C2','C4','C6','CP4','CP2',]
info_motor = mne.create_info(ch_names=info_motorbrain, ch_types="eeg", sfreq=1024)
info_motor.set_montage('standard_1020')

In [ ]:
#train testの分割
X_train, X_test, Y_train, Y_test = train_test_split(data_numpy_task,label_data, test_size=0.25,random_state=42)
print(np.shape(X_train))

In [ ]:
#default CSP
# CSP(n_components=4, reg=None, log=None, cov_est='concat', transform_into='average_power'
#     , norm_trace=False, cov_method_params=None, rank=None, component_order='mutual_info')

In [ ]:
csp = CSP(n_components=8, reg='empirical', log=True, cov_est='concat', transform_into='average_power', 
          norm_trace=True, cov_method_params=None, rank='info', component_order='alternate')

In [ ]:
# plot CSP patterns estimated on full data for visualization
kenshou = csp.fit_transform(X_train, Y_train)
csp.fit_transform(X_train, Y_train)
csp.plot_patterns(info_motor,ch_type='eeg', units='None', size=1.5)

In [ ]:
x_train_csp = csp.transform(X_train)
x_test_csp = csp.transform(X_test)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
# 多層パーセプトロン
mlp_clf = MLPClassifier()
mlp_clf.fit(x_train_csp, Y_train)

mlp_clf.score(x_test_csp, Y_test)

In [ ]:
random_search = {'batch_size': [10, 20, 30, 130],
               'hidden_layer_sizes': [(150, 150), (100, 100, 50), (150, 100, 50)],
               'max_iter': [400, 500, 600, 700, 800],
               'random_state': [42]}
mlp_random_search2 = RandomizedSearchCV(mlp_clf, random_search, cv=3,
                                   n_jobs=-1, scoring='accuracy', random_state=0)
# スケーリングしたデータで実行します
mlp_random_search2.fit(x_train_csp, Y_train)

In [ ]:
# 実行
cvres = mlp_random_search2.cv_results_
for score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(score, params)

In [ ]:
mlp_random_search2.score(x_test_csp, Y_test)

In [ ]:
#時系列データのSVM
model =SVC(kernel='linear')
model.fit(x_train_csp,Y_train)
print(model.score(x_test_csp, Y_test))

In [26]:
params = {
    "C":np.logspace(-1,10,12),
    "kernel":["rbf","linear","sigmoid"],
    "gamma":np.logspace(-8,3,12)
}
grid = GridSearchCV(model, params,scoring="accuracy", cv=5)

grid.fit(x_train_csp,Y_train)

print("grid_best_score=",grid.best_score_)
print(grid.best_params_)

pred = grid.predict(x_test_csp)
print(classification_report(Y_test, pred))